In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
from pl_models import RNNModel, LSTMModel, GRUModel
from pl_config import stock_ticker, company_name, num_units, num_layers, dropout_prob, architecture, seq_length, start_date, end_date, num_epochs, learning_rate
import wandb


class StockPredictionModule(pl.LightningModule):
    def __init__(self, model, scaler, train_loader, val_loader, test_loader):
        super().__init__()
        self.model = model
        self.scaler = scaler
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.criterion = nn.MSELoss()

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        return optimizer

    def training_step(self, batch, batch_idx):
        seqs, labels = batch
        y_pred = self(seqs)
        loss = self.criterion(y_pred, labels)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        seqs, labels = batch
        y_pred = self(seqs)
        loss = self.criterion(y_pred, labels)
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        seqs, labels = batch
        y_pred = self(seqs)
        loss = self.criterion(y_pred, labels)
        self.log("test_loss", loss)
        return loss

    def on_test_epoch_end(self):
        predictions, actuals = [], []
        for seqs, labels in self.test_loader:
            seqs, labels = seqs.to(self.device), labels.to(self.device)
            output = self(seqs)
            predictions.extend(output.view(-1).tolist())
            actuals.extend(labels.view(-1).tolist())
        
        predictions = np.array(predictions)
        actuals = np.array(actuals)
        predictions_original_scale = self.scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()
        actuals_original_scale = self.scaler.inverse_transform(actuals.reshape(-1, 1)).flatten()
        
        plt.figure(figsize=(10, 5))
        plt.plot(actuals_original_scale, label='Actual Price', color='blue')
        plt.plot(predictions_original_scale, label='Predicted Price', color='red', linestyle='--')
        plt.title('Stock Price Prediction')
        plt.xlabel('Time')
        plt.ylabel('Stock Price')
        plt.legend()
        plt.tight_layout()
        
        wandb.log({"Predictions vs Actuals": wandb.Image(plt)})
        plt.close()


def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data)-seq_length-1):
        xs.append(data[i:(i+seq_length)])
        ys.append(data[i+seq_length])
    return np.array(xs), np.array(ys)


def load_data():
    stock_df = yf.download(stock_ticker, start=start_date, end=end_date)

    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_data = scaler.fit_transform(stock_df[['Close']].values.reshape(-1, 1))
    X, y = create_sequences(scaled_data, seq_length)
    
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    train_loader = DataLoader(TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train)), batch_size=64, shuffle=False)
    val_loader = DataLoader(TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val)), batch_size=64, shuffle=False)
    test_loader = DataLoader(TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test)), batch_size=64, shuffle=False)
    
    return train_loader, val_loader, test_loader, scaler


def main():

    config = {
        "dataset": f"{company_name} closing prices",
        "architecture": architecture,
        "num_units": num_units,
        "num_layers": num_layers,
        "dropout": dropout_prob,
        "seq_length": seq_length,
        "start_date": start_date,
        "end_date": end_date,
        "epochs": num_epochs,
        "learning_rate": learning_rate
    }

    wandb_logger = WandbLogger(project="RNN_single_step_forecasts", log_model="all", config=config)

    train_loader, val_loader, test_loader, scaler = load_data()
    
    model_config = {
        "input_size": 1,       
        "hidden_layer_size": num_units,   
        "num_layers": num_layers,           
        "output_size": 1,          
        "dropout_prob": dropout_prob
    }

    if architecture == "RNN":
        model = RNNModel(**model_config)
    elif architecture == "LSTM":
        model = LSTMModel(**model_config)
    elif architecture == "GRU":
        model = GRUModel(**model_config)
    else:
        raise ValueError("Unsupported architecture specified")

    module = StockPredictionModule(model=model, scaler=scaler, train_loader=train_loader, val_loader=val_loader, test_loader=test_loader)

    # Device agnostic initialization
    if torch.cuda.is_available():   # Check for GPU availability
        accelerator = "gpu"
        devices = 1
    elif hasattr(torch, 'has_mps') and torch.backends.mps.is_built():  # Check for MPS availability (Apple Silicon)
        accelerator = "mps"
        devices = 1
    else:
        accelerator = None  # Defaults to CPU
        devices = None  # Ignored for CPU

    trainer = Trainer(max_epochs=num_epochs, logger=wandb_logger, accelerator=accelerator, devices=devices, enable_checkpointing=True)
    trainer.fit(module, train_dataloaders=train_loader, val_dataloaders=val_loader)
    trainer.test(dataloaders=test_loader, ckpt_path="best")

    wandb.finish()


if __name__ == "__main__":
    main()



[*********************100%%**********************]  1 of 1 completed
/var/folders/72/xqxvrz7s4n7grqqwprj1mrc80000gn/T/ipykernel_5962/3972846076.py:151: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  elif hasattr(torch, 'has_mps') and torch.has_mps:  # Check for MPS availability (Apple Silicon)
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


RNN


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: frederik135. Use `wandb login --relogin` to force relogin



  | Name      | Type     | Params
---------------------------------------
0 | model     | RNNModel | 1.8 K 
1 | criterion | MSELoss  | 0     
---------------------------------------
1.8 K     Trainable params
0         Non-trainable params
1.8 K     Total params
0.007     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=150` reached.
Restoring states from the checkpoint path at ./RNN_single_step_forecasts/nm1lkojn/checkpoints/epoch=149-step=7050.ckpt
Loaded model weights from the checkpoint at ./RNN_single_step_forecasts/nm1lkojn/checkpoints/epoch=149-step=7050.ckpt
/opt/anaconda3/envs/masterarbeit/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss          0.0005179913714528084
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_loss,▁
train_loss_epoch,█▆▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▃▅▂▃▃▃▂▂▄▂▂▄▂▂▁▂▂▁▂▁▂▂▂▁▁▁▄▁▂▁▂▂▁▂▁▁▁▁▂
trainer/global_step,▁▁▁▁▁▁▁▂▁▃▁▃▁▁▁▁▄▁▄▁▅▁▂▂▂▆▂▆▂▂▂▂▂▂█▂█▂▂▂
val_loss_epoch,█▅▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_step,█▅▄▃▃▂▂▃▃▃▃▃▂▂▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂
epoch,150
test_loss,0.00052
train_loss_epoch,0.00034
train_loss_step,0.00029
